In [129]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


import pickle
import pandas as pd
import numpy as np
from IPython.display import clear_output
import time
import random
import matplotlib.pyplot as plt

# Import data and fix up for transformer

In [130]:
# Replace 'your_file.pkl' with the path to your pickle file
file_path = "Connect4Dataset_RandomOpp_Firstmove.pkl"

# Open the pickle file in read mode
with open(file_path, "rb") as file:
    dataset_progressive_skill = pickle.load(file)

# Convert the dataset to a DataFrame, including board state
data = []
for record in dataset_progressive_skill:
    game_id, board, move, player1_skill, player2_skill, player = record
    data.append(
        {
            "Game ID": game_id,
            "Board State": board,
            "Move": move,
            "Player 1 Skill": player1_skill,
            "Player 2 Skill": player2_skill,
            "Player": player,
        }
    )

connect4 = pd.DataFrame(data)

In [131]:
def standardize_board_state(board, player):
    """
    Swap [0,1] to [1,0] and [1,0] to [0,1] in a given board only if player == -1.
    """

    # if isinstance(player, str):
    #     player = int(player)

    if player == "minus":
        board = np.array(board)  # Ensure it's a NumPy array

        # Create masks
        mask_01 = (board[:, :, 0] == 0) & (board[:, :, 1] == 1)  # Find [0,1]
        mask_10 = (board[:, :, 0] == 1) & (board[:, :, 1] == 0)  # Find [1,0]

        # Swap values
        board[mask_01] = [1, 0]
        board[mask_10] = [0, 1]

    return board  # Return the modified or original board


# Apply to each board where the player is -1
connect4_standardized = connect4.copy()
connect4_standardized["Board State"] = connect4_standardized.apply(
    lambda row: standardize_board_state(row["Board State"], row["Player"]), axis=1
)

In [132]:
def flip_board(board):
    """Flip the board horizontally."""
    return np.flip(board, axis=1)


def add_flipped_boards(df):
    """
    Add flipped boards to the dataset, updating the Game ID for flipped boards.
    Args:
        df: A Pandas DataFrame with columns "Game ID", "Board State", "Move",
             "Player 1 Skill", "Player 2 Skill", and "Player".
    Returns:
        A new DataFrame with original and flipped boards, where flipped boards have updated Game IDs.
    """
    flipped_rows = []
    max_game_id = df[
        "Game ID"
    ].max()  # Start new Game IDs after the max ID in the original DataFrame

    for game_id, group in df.groupby("Game ID"):  # Group by each game
        for _, row in group.iterrows():
            # Extract board state, move, and other columns
            board = row["Board State"]
            move = row["Move"]
            other_columns = row.drop(
                ["Game ID", "Board State", "Move"]
            ).to_dict()  # Extract other columns

            # Add the original board state
            flipped_rows.append(
                {
                    "Game ID": game_id,  # Keep the original Game ID
                    "Board State": board,
                    "Move": move,
                    **other_columns,
                }
            )

            # Flip the board and adjust the move
            flipped_board = flip_board(board)
            flipped_move = 6 - move  # Adjust move for flipped board

            # Add the flipped board state with a new Game ID
            flipped_rows.append(
                {
                    "Game ID": max_game_id
                    + 1,  # Increment Game ID for flipped board states
                    "Board State": flipped_board,
                    "Move": flipped_move,
                    **other_columns,
                }
            )

        # Increment the max_game_id for the next game
        max_game_id += 1

    # Create a new DataFrame from the rows
    return pd.DataFrame(flipped_rows)


# Apply the function to your DataFrame
connect4_wflip = add_flipped_boards(connect4_standardized)
connect4_wflip.shape

(605018, 6)

# Transformer architecture Dans code

In [154]:
class PositionalIndex(tf.keras.layers.Layer):
    def call(self, x):
        bs = tf.shape(x)[0]  # extract batch size
        number_of_vectors = tf.shape(x)[
            1
        ]  # how many vectors - we know it should be m*n, but let's count to be sure...
        indices = tf.range(number_of_vectors)  # index for each vector
        indices = tf.expand_dims(indices, 0)  # reshape it appropriately
        return tf.tile(indices, [bs, 1])  # repeat for each batch

In [155]:
class ClassTokenIndex(tf.keras.layers.Layer):
    def call(self, x):
        bs = tf.shape(x)[0]  # extract batch size
        number_of_vectors = 1  # how many vectors - we just want 1 (which is @ 0) ... we only want to generate 1 vector for the class token
        # now just get it to be the right size
        indices = tf.range(number_of_vectors)  # index for each vector
        indices = tf.expand_dims(indices, 0)  # reshape it appropriately
        return tf.tile(indices, [bs, 1])  # repeat for each batch

In [156]:
def build_ViT(
    n,
    m,
    block_size,
    hidden_dim,
    num_layers,
    num_heads,
    key_dim,
    value_dim,
    mlp_dim,
    dropout_rate,
    num_classes,
):
    # n is number of rows of blocks
    # m is number of cols of blocks
    # block_size is number of pixels (with rgb) in each block
    inp = tf.keras.layers.Input(shape=(n * m, block_size))
    mid = tf.keras.layers.Dense(hidden_dim)(
        inp
    )  # transform to vectors with different dimension
    # the positional embeddings
    inp2 = PositionalIndex()(inp)
    emb = tf.keras.layers.Embedding(input_dim=n * m, output_dim=hidden_dim)(
        inp2
    )  # learned positional embedding for each of the n*m possible possitions
    mid = tf.keras.layers.Add()(
        [mid, emb]
    )  # for some reason, tf.keras.layers.Add causes an error, but + doesn't?
    # create and append class token to beginning of all input vectors
    tokenInd = ClassTokenIndex()(mid)
    token = tf.keras.layers.Embedding(input_dim=1, output_dim=hidden_dim)(tokenInd)
    mid = tf.keras.layers.Concatenate(axis=1)([token, mid])

    for l in range(num_layers):  # how many Transformer Head layers are there?
        ln = tf.keras.layers.LayerNormalization()(mid)  # normalize
        mha = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=key_dim, value_dim=value_dim
        )(
            ln, ln, ln
        )  # self attention!
        add = tf.keras.layers.Add()([mid, mha])  # add and norm
        ln = tf.keras.layers.LayerNormalization()(add)
        den = tf.keras.layers.Dense(mlp_dim, activation="gelu")(
            ln
        )  # maybe should be relu...who knows...
        den = tf.keras.layers.Dropout(dropout_rate)(den)  # regularization
        den = tf.keras.layers.Dense(hidden_dim)(
            den
        )  # back to the right dimensional space
        den = tf.keras.layers.Dropout(dropout_rate)(den)
        mid = tf.keras.layers.Add()([den, add])  # add and norm again

    fl = mid[:, 0, :]  # just grab the class token for each image in batch
    ln = tf.keras.layers.LayerNormalization()(fl)
    clas = tf.keras.layers.Dense(num_classes, activation="softmax")(
        ln
    )  # probability that the image is in each category
    mod = tf.keras.models.Model(inp, clas)
    mod.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return mod

In [157]:
n = 6
m = 7
block_size = 2
hidden_dim = 64
num_layers = 12
num_heads = 12
key_dim = (
    hidden_dim // num_heads
)  # usually good practice for key_dim to be hidden_dim//num_heads...this is why we do Multi-Head attention
value_dim = key_dim * 2
mlp_dim = hidden_dim
dropout_rate = 0.05
num_classes = 7


trans = build_ViT(
    n,
    m,
    block_size,
    hidden_dim,
    num_layers,
    num_heads,
    key_dim,
    value_dim,
    mlp_dim,
    dropout_rate,
    num_classes,
)
trans.summary()

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_22      │ (None, 42, 2)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_index_16 │ (None, 42)        │          0 │ input_layer_22[0… │
│ (PositionalIndex)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_304 (Dense)   │ (None, 42, 64)    │        192 │ input_layer_22[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_32        │ (None, 42, 64)    │      2,688 │ positional_index… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_280 (Add)       │ (None, 42, 64)    │          0 │ dense_304[0][0],  │
│                     │                   │            │ embedding_32[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ class_token_index_… │ (None, 1)         │          0 │ add_280[0][0]     │
│ (ClassTokenIndex)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_33        │ (None, 1, 64)     │         64 │ class_token_inde… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_14      │ (None, 43, 64)    │          0 │ embedding_33[0][… │
│ (Concatenate)       │                   │            │ add_280[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 43, 64)    │        128 │ concatenate_14[0… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 43, 64)    │     23,344 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_281 (Add)       │ (None, 43, 64)    │          0 │ concatenate_14[0… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 43, 64)    │        128 │ add_281[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_305 (Dense)   │ (None, 43, 64)    │      4,160 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_399         │ (None, 43, 64)    │          0 │ dense_305[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_306 (Dense)   │ (None, 43, 64)    │      4,160 │ dropout_399[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_400         │ (None, 43, 64)    │          0 │ dense_306[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_282 (Add)       │ (None, 43, 64)    │          0 │ dropout_400[0][0… │
│                     │                   │            │ add_281[0][0]     │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 386,567 (1.47 MB)

 Trainable params: 386,567 (1.47 MB)

 Non-trainable params: 0 (0.00 B)

In [158]:
(
    ((hidden_dim * key_dim + key_dim) * 2 + hidden_dim * value_dim + value_dim)
    * num_heads
    + (value_dim * num_heads) * hidden_dim
    + hidden_dim
)

23344

# Prep data for testing

In [165]:
def subset_games(df, num_games):
    """
    Subset a DataFrame based on a random sample of unique game IDs.

    Parameters:
        df (pd.DataFrame): The input DataFrame with a 'game id' column.
        num_games (int): The number of unique game IDs to subset.

    Returns:
        pd.DataFrame: A subset of the original DataFrame.
    """
    # Get the unique game IDs
    unique_game_ids = df["Game ID"].unique()

    # Ensure the requested number of games doesn't exceed the available unique IDs
    if num_games > len(unique_game_ids):
        raise ValueError(
            f"num_games ({num_games}) exceeds the total unique game IDs ({len(unique_game_ids)})."
        )

    # Randomly sample the desired number of unique game IDs
    sampled_game_ids = pd.Series(unique_game_ids).sample(num_games, random_state=42)

    # Subset the DataFrame
    subset_df = df[df["Game ID"].isin(sampled_game_ids)]

    return subset_df


num_games = 20000  # Specify the number of games to include
connect4_testing = subset_games(connect4_wflip, num_games)

# Print some details
print(f"Subset contains {len(connect4_testing)} board states from {num_games} games.")

Subset contains 605018 board states from 20000 games.


In [127]:
# Extract features (board states) and labels (recommended moves)
X = np.stack(
    connect4_testing["Board State"].values
)  # Convert board states into a NumPy array
y = connect4_testing["Move"].values

# Normalize the board states (optional for CNNs)
X = X.astype("float32") / 1.0  # Assuming board states are binary (0 or 1)

# Convert labels to one-hot encoding (required for multi-class classification)
num_classes = 7  # Moves are in columns 0-6
y = to_categorical(y, num_classes)

# Split into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

print(
    f"Training set: {X_train.shape}, Validation set: {X_val.shape}, Test set: {X_test.shape}"
)


ndata_train = X_train.shape[0]
ndata_test = X_test.shape[0]

Training set: (423512, 6, 7, 2), Validation set: (90753, 6, 7, 2), Test set: (90753, 6, 7, 2)


In [117]:
y_train = np.argmax(y_train, axis=1)  # Convert from one-hot to class indices
y_test = np.argmax(y_test, axis=1)  # Convert from one-hot to class indices

In [118]:
x_train_ravel = np.zeros((ndata_train, n * m, block_size))
for img in range(ndata_train):
    ind = 0
    for row in range(n):
        for col in range(m):
            x_train_ravel[img, ind, :] = X_train[
                img,
                row,
                col,
                :,
                # img, (row * 4) : ((row + 1) * 4), (col * 4) : ((col + 1) * 4)
            ].ravel()
            ind += 1

In [119]:
x_train_ravel.shape

(423512, 42, 2)

In [120]:
x_test_ravel = np.zeros((ndata_test, n * m, block_size))
for img in range(ndata_test):
    ind = 0
    for row in range(n):
        for col in range(m):
            x_test_ravel[img, ind, :] = X_test[
                img,
                row,
                col,
                :,
                # img, (row * 4) : ((row + 1) * 4), (col * 4) : ((col + 1) * 4)
            ].ravel()
            ind += 1

In [136]:
# # Extract board states and moves
# X = np.array(connect4_wflip["Board State"].tolist())  # Convert to NumPy array
# y = np.array(connect4_wflip["Move"])  # Target moves (0 to 6)

# # Verify shapes
# print("Board State Shape:", X.shape)  # Expecting (num_samples, 6, 7, 2)
# print("Move Labels Shape:", y.shape)  # Expecting (num_samples,)

# # Convert moves to one-hot encoding (since we have 7 possible moves)
# y_onehot = tf.keras.utils.to_categorical(y, num_classes=7)

# # Split into train/test sets
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y_onehot, test_size=0.2, random_state=42
# )

# print("Training Shape:", X_train.shape, y_train.shape)
# print("Test Shape:", X_test.shape, y_test.shape)

Board State Shape: (605018, 6, 7, 2)
Move Labels Shape: (605018,)
Training Shape: (484014, 6, 7, 2) (484014, 7)
Test Shape: (121004, 6, 7, 2) (121004, 7)


### Train 

In [ ]:
# trans.fit(x_train_ravel, y_train, epochs=1, batch_size=40, validation_split=0.15) # Dans
# history = transformer.fit(
#     X_train,
#     y_train,
#     validation_data=(X_test, y_test),
#     epochs=1,  # Increase for better performance
#     batch_size=32,
# )  # Not Dans

15126/15126 ━━━━━━━━━━━━━━━━━━━━ 392s 26ms/step - accuracy: 0.2191 - loss: 1.8890 - val_accuracy: 0.2249 - val_loss: 1.8710


In [128]:
out = trans.evaluate(x_test_ravel, y_test) # Dans

In [ ]:
out[1] # Dans

# Evaluate on test set
# test_loss, test_acc = transformer.evaluate(X_test, y_test)
# print(f"Test Accuracy: {test_acc:.4f}")

3782/3782 ━━━━━━━━━━━━━━━━━━━━ 36s 10ms/step - accuracy: 0.2225 - loss: 1.8723
Test Accuracy: 0.2249


In [144]:
# trans.save("connect4_transformer.h5") Dans
trans.save("connect4_transformer_test.h5") 

# Save the trained model
# transformer.save("connect4_transformer_test.h5")